In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
from nltk.util import bigrams,trigrams
from fuzzywuzzy import fuzz
from collections import defaultdict
import re

nltk.download('punkt')
nltk.download('stopwords')

/home/zafir/anaconda3/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
[nltk_data] Downloading package punkt to /home/zafir/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/zafir/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
pd.set_option('display.max_colwidth', None)
tweet_dataset='gg2013.json/gg2013.json'
df=pd.read_json(tweet_dataset)

In [3]:
df

,text,user,id,timestamp_ms
0,JLo's dress! #eredcarpet #GoldenGlobes,"{'screen_name': 'Dozaaa_xo', 'id': 557374298}",290620657987887104,2013-01-14 00:45:38
1,What's making Sofia Vergara's boobs stay like that? Magic? Witchcraft? #GoldenGlobes,"{'screen_name': 'theAmberShow', 'id': 14648726}",290620657887219713,2013-01-14 00:45:38
2,RT @FabSugar: Kerry Washington is EVERYTHING. Dying over her Miu Miu gown! @goldenglobes #goldenglobes http://t.co/3feH7MvO,"{'screen_name': 'SweetyPW', 'id': 35498686}",290620657828524032,2013-01-14 00:45:38
3,Anne Hathaway has got me living. #GoldenGlobes,"{'screen_name': '_NicoleEdwards', 'id': 144430208}",290620657799159809,2013-01-14 00:45:38
4,Jennifer Lopez's lace dress? Thoughts? #GoldenGlobes,"{'screen_name': 'lolaogunnaike', 'id': 134953223}",290620657778188288,2013-01-14 00:45:38
...,...,...,...,...
174638,RT @authorViviAnna: I was sad that Mandy Patinkin didn't win #GoldenGlobes he is so good in Homeland,"{'screen_name': 'dana1204', 'id': 18091543}",290675889379876864,2013-01-14 04:25:07
174639,RT @_ItzelMartinez_: Jennifer Lawrence aceptando su premio #GoldenGlobes ❤\nhttp://t.co/c1xqCz4W,"{'screen_name': 'IamTrisEverdeen', 'id': 551264918}",290675889128230914,2013-01-14 04:25:07
174640,"Golden Globes, lots of fashion messes...but glad Argo won. Really good movie, along w Moonrise Kingdom and Salmon Fishing Yeman.","{'screen_name': 'Dpharmakis23', 'id': 852045842}",290675893024747523,2013-01-14 04:25:08
174641,Did they have mug shots at the golden globes?! DID THEY?! #localnews http://t.co/KA8Nsh2h,"{'screen_name': 'reynaramirez22', 'id': 22732662}",290675888763314178,2013-01-14 04:25:08


In [4]:
def filter_rows_by_keywords(df, column_name, keywords):
    
    pattern = '|'.join(keywords)
    return df[df[column_name].str.contains(pattern, case=False, na=False)]

In [5]:
keywords=['best']
filtered_df = filter_rows_by_keywords(df, 'text', keywords)
filtered_df.head()

,text,user,id,timestamp_ms
16,Why did I just flip out? Because one of the best people in the world showed up on the tv..Kristen Wiig #goldenglobes !,"{'screen_name': 'mo_JOE_marino', 'id': 286809109}",290620657170014208,2013-01-14 00:45:38
47,"Seacrest asked what Ben Affleck's kids wrote on his hands. Best use of ""I'd tell you but I'd have to kill you."" #GoldenGlobes #redcarpet","{'screen_name': 'StaceyWOMC', 'id': 518510715}",290620662043779072,2013-01-14 00:45:39
49,"RT @Loki_Page: #GoldenGlobes Times! We wish good luck tonight to Rachel Weisz, nominated for Best Actress. ""The Deep Blue Sea""","{'screen_name': 'QueenF1989', 'id': 117283451}",290620661959888896,2013-01-14 00:45:39
67,Blinded by Kerry Washington's Miu Miu dress and her gorgeous bling. We are in LOVE. #GoldenGlobes #bestdressed,"{'screen_name': 'snobswap', 'id': 247506902}",290620659837587456,2013-01-14 00:45:39
130,"Seacrest asked what Ben Affleck's kids wrote on his hands. Best use of ""I'd tell you but I'd have to kill you."" #GoldenGlobes #redcarpet","{'screen_name': 'staceyduford', 'id': 15548531}",290620663486631937,2013-01-14 00:45:40


In [6]:
#bigram or trigram, variable written as bigram only

text = " ".join(row for row in filtered_df['text']).lower()


tokens = word_tokenize(text)


filtered_tokens = [word for word in tokens if word.isalnum() and word not in stopwords.words('english')]


bigram_list = list(trigrams(filtered_tokens))

best_bigrams = [bg for bg in bigram_list if bg[0] == "best"]


best_bigram_counts = Counter(best_bigrams)


top_best_bigrams = best_bigram_counts.most_common(26)

print(top_best_bigrams)

[(('best', 'supporting', 'actress'), 1302), (('best', 'motion', 'picture'), 1203), (('best', 'original', 'song'), 1033), (('best', 'actor', 'motion'), 722), (('best', 'supporting', 'actor'), 698), (('best', 'actress', 'motion'), 617), (('best', 'actress', 'tv'), 454), (('best', 'actor', 'tv'), 385), (('best', 'tv', 'series'), 347), (('best', 'director', 'ben'), 337), (('best', 'television', 'series'), 323), (('best', 'animated', 'feature'), 323), (('best', 'original', 'score'), 278), (('best', 'actress', 'comedy'), 228), (('best', 'foreign', 'film'), 189), (('best', 'screenplay', 'quentin'), 177), (('best', 'golden', 'globes'), 160), (('best', 'hosts', 'ever'), 156), (('best', 'acceptance', 'speech'), 156), (('best', 'speech', 'ever'), 150), (('best', 'actor', 'drama'), 139), (('best', 'animated', 'film'), 136), (('best', 'comedy', 'series'), 134), (('best', 'director', 'argo'), 128), (('best', 'director', 'goldenglobes'), 117), (('best', 'buy', 'demonstrate'), 116)]


In [7]:
df_movie_actor=pd.read_json('gg2013_actor_movie.json')
df_movie_actor

,text,user,id,timestamp_ms,movie,name,NLTK
0,What's making Sofia Vergara's boobs stay like that? Magic? Witchcraft? #GoldenGlobes,"{'screen_name': 'theAmberShow', 'id': 14648726}",290620657887219713,2013-01-14 00:45:38,"[[13, Making], [19, Sofia], [27, Sofia Vergara], [35, BOOBS], [57, Magic], [64, Witch], [69, Witchcraft]]","[[27, Sofia Vergara]]","[Sofia, Vergara, Magic, Witchcraft, GoldenGlobes]"
1,RT @FabSugar: Kerry Washington is EVERYTHING. Dying over her Miu Miu gown! @goldenglobes #goldenglobes http://t.co/3feH7MvO,"{'screen_name': 'KaterinaSpirot', 'id': 422770788}",290643811170918401,2013-01-14 02:17:39,"[[19, Kerry], [27, Washing], [30, Kerry Washington], [30, Washington], [44, Everything]]","[[30, Kerry Washington], [68, Miu Miu]]","[RT, @, FabSugar, Kerry, Washington, EVERYTHING, Miu, Miu]"
2,Anne Hathaway has got me living. #GoldenGlobes,"{'screen_name': '_NicoleEdwards', 'id': 144430208}",290620657799159809,2013-01-14 00:45:38,"[[13, Anne Hathaway], [31, Living]]","[[13, Anne Hathaway]]","[Anne, Hathaway, GoldenGlobes]"
3,Jennifer Lopez's lace dress? Thoughts? #GoldenGlobes,"{'screen_name': 'lolaogunnaike', 'id': 134953223}",290620657778188288,2013-01-14 00:45:38,"[[5, Jenni], [8, Jennifer], [14, Jennifer Lopez], [37, Thoughts]]","[[14, Jennifer Lopez]]","[Jennifer, Lopez, GoldenGlobes]"
4,"RT @BillMc7: ""Wait. What's that smell?!"" (everyone being interviewed by Al Roker) #GoldenGlobes","{'screen_name': 'xtinavickers', 'id': 92139052}",290620657715253248,2013-01-14 00:45:38,"[[38, What's That Smell?], [56, Being], [66, Interview], [80, Al Roker]]","[[80, Al Roker]]","[RT, @, BillMc7, Wait, Al, Roker, GoldenGlobes]"
...,...,...,...,...,...,...,...
61618,@goldenglobes \n\nSilver Linings Superior to Les Miss by. Yard. You got not wrong,"{'screen_name': 'Imused', 'id': 294324510}",290675890818527232,2013-01-14 04:25:07,"[[36, Super], [50, Les Mis], [80, Wrong]]","[[53, Miss B]]","[Silver, Linings, Superior, Les, Miss, Yard]"
61619,"""@nbcsnl: Just thinking ahead, but like should Will Ferrell and Kristen Wiig host the #GoldenGlobes together next year??"" BOOM!!!","{'screen_name': 'NickAspel', 'id': 291493551}",290675889837051904,2013-01-14 04:25:07,"[[20, Think], [59, Will Ferrell], [70, Kriste], [71, Kristen], [76, Kristen Wiig], [108, Together], [127, Boom!]]","[[76, Kristen Wiig], [59, Will Ferrell]]","[Will, Ferrell, Kristen, Wiig, GoldenGlobes]"
61620,Ben Affleck celebrates his win backstage. #goldenglobes http://t.co/BDuLABKZ // les dije qe fueran a verla! :),"{'screen_name': 'Mer_Contreras', 'id': 35919432}",290675889556041728,2013-01-14 04:25:07,"[[11, Ben Affleck], [21, Celebrate], [40, Backstage]]","[[11, Ben Affleck]]","[Ben, Affleck]"
61621,RT @authorViviAnna: I was sad that Mandy Patinkin didn't win #GoldenGlobes he is so good in Homeland,"{'screen_name': 'dana1204', 'id': 18091543}",290675889379876864,2013-01-14 04:25:07,"[[40, Mandy], [49, Mandy Patinkin], [100, Homeland]]","[[49, Mandy Patinkin]]","[RT, @, Mandy, Patinkin, GoldenGlobes, Homeland]"


In [8]:
bigram_patterns = [bigram for bigram, count in top_best_bigrams]

dataframe_actor_or_filtered=df_movie_actor

def contains_top_bigram(text):
    tokens = word_tokenize(text.lower())
    text_bigrams = list(trigrams(tokens))
    return any(bg in text_bigrams for bg in bigram_patterns)

filtered_df_best_words = dataframe_actor_or_filtered[dataframe_actor_or_filtered['text'].apply(contains_top_bigram)]

In [9]:
filtered_df_best_words

,text,user,id,timestamp_ms,movie,name,NLTK
663,RT @TheAdvocateMag: Modern Family is up for best TV comedy and whose Eric Stonestreet (who plays gay dad Cam) is up for best supporting actor. #GoldenGlobes,"{'screen_name': 'sandrasick', 'id': 19883537}",290621553153036289,2013-01-14 00:49:12,"[[33, Modern Family], [33, Family], [58, Comedy], [79, Stone], [80, Stones], [85, Eric Stonestreet], [132, Support], [141, Best Supporting Actor], [141, Actor]]","[[85, Eric Stonestreet]]","[RT, @, TheAdvocateMag, Family, Eric, Stonestreet, Cam, GoldenGlobes]"
890,RT @COVERGIRL: Our fingers are crossed! #goldenglobes RT @savvysassymoms: Congrats to @COVERGIRL Sofia Vergara – Best Supporting Actress #PGRedCarpet,"{'screen_name': 'HannaGillispie', 'id': 62957506}",290622018582360064,2013-01-14 00:51:03,"[[26, Fingers], [36, Cross], [38, Crossed], [82, Congrats], [102, Sofia], [110, Sofia Vergara], [125, Support], [136, Actress]]","[[110, Sofia Vergara]]","[RT, @, COVERGIRL, RT, COVERGIRL, Sofia, Vergara, –, Best, Supporting, Actress, PGRedCarpet]"
1161,"It's time for the best Golden Globes ever! @NotTinaFey &amp; Amy Poehler are gonna kill it! Lets go @bekgaston, @amanda_wongg &amp; @JessicaTong!","{'screen_name': 'LMae13', 'id': 24568030}",290622626697736192,2013-01-14 00:53:28,"[[9, It's Time], [22, For the Best], [22, The Best], [29, Golden], [36, Golden Globes], [72, Amy Poehler]]","[[72, Amy Poehler]]","[Globes, @, NotTinaFey, Amy, Poehler, @, JessicaTong]"
1501,@GoldenGlobes Anne Hathaway best supporting Actress &gt;&gt;&gt;,"{'screen_name': 'DerekBiaselli', 'id': 349959219}",290623059939971072,2013-01-14 00:55:11,"[[27, Anne Hathaway], [40, Support], [51, Actress]]","[[27, Anne Hathaway]]","[GoldenGlobes, Anne, Hathaway, Actress]"
1528,RT @ANCALERTS: MT @LexiSchulze Best supporting actress nominee Sofia Vergara is rocking a beautiful black Michael Cinco number at the #GoldenGlobes!,"{'screen_name': 'yeohrocks', 'id': 38415624}",290656197793697792,2013-01-14 03:06:52,"[[43, Support], [54, Actress], [68, Sofia], [76, Sofia Vergara], [99, Beautiful], [105, Black], [113, Michael], [119, Cinco], [126, Number]]","[[76, Sofia Vergara], [119, Michael Cinco]]","[RT, @, ANCALERTS, MT, @, LexiSchulze, Best, Sofia, Vergara, Michael, Cinco, GoldenGlobes]"
...,...,...,...,...,...,...,...
61178,Best Golden Globes ever. So much love for Adele and Jennifer Lawrence #GirlCrush,"{'screen_name': 'MinaKokic', 'id': 46554204}",290675665471160321,2013-01-14 04:24:13,"[[11, Golden], [18, Golden Globes], [47, Adele], [57, Jenni], [60, Jennifer], [69, Jennifer Lawrence], [69, Lawrence]]","[[69, Jennifer Lawrence], [47, Adele]]","[Best, Golden, Globes, Adele, Jennifer, Lawrence, GirlCrush]"
61197,"RT @SandraOldfield: Ironic that the best acceptance speech for an acting awards show came from a singer, Adele #goldenglobes","{'screen_name': 'jeffbruchesi', 'id': 208374605}",290675667958370304,2013-01-14 04:24:14,"[[40, The Best], [47, Accept], [51, Acceptance], [58, Speech], [72, Acting], [79, Awards], [103, Singer], [110, Adele]]","[[110, Adele]]","[RT, @, SandraOldfield, Adele]"
61205,"""@goldenglobes: Best Original Song - ""Skyfall"" - Adele (@OfficialAdele) &amp; Paul Epworth (@PaulEpworth) - Skyfall - #GoldenGlobes"" :o","{'screen_name': 'radarkepo', 'id': 141178894}",290675671863283712,2013-01-14 04:24:15,"[[27, Origin], [29, Original], [34, Original Song], [54, Adele], [115, Skyfall]]","[[90, Paul Epworth], [54, Adele]]","[Best, Original, Song, Adele, @, OfficialAdele, Paul, Epworth, @, PaulEpworth, GoldenGlobes]"
61252,"RT @televisionary: Best Supporting Actress, TV goes to… Maggie Smith. As always. #GoldenGlobes","{'screen_name': 'edgicovi', 'id': 44584283}",290675714229944320,2013-01-14 04:24:25,"[[31, Support], [42, Actress], [61, Maggi], [62, Maggie], [68, Smith], [79, Always]]","[[68, Maggie Smith]]","[RT, @, Actress, Maggie, Smith, GoldenGlobes]"


In [10]:
filtered_df_best_words

,text,user,id,timestamp_ms,movie,name,NLTK
663,RT @TheAdvocateMag: Modern Family is up for best TV comedy and whose Eric Stonestreet (who plays gay dad Cam) is up for best supporting actor. #GoldenGlobes,"{'screen_name': 'sandrasick', 'id': 19883537}",290621553153036289,2013-01-14 00:49:12,"[[33, Modern Family], [33, Family], [58, Comedy], [79, Stone], [80, Stones], [85, Eric Stonestreet], [132, Support], [141, Best Supporting Actor], [141, Actor]]","[[85, Eric Stonestreet]]","[RT, @, TheAdvocateMag, Family, Eric, Stonestreet, Cam, GoldenGlobes]"
890,RT @COVERGIRL: Our fingers are crossed! #goldenglobes RT @savvysassymoms: Congrats to @COVERGIRL Sofia Vergara – Best Supporting Actress #PGRedCarpet,"{'screen_name': 'HannaGillispie', 'id': 62957506}",290622018582360064,2013-01-14 00:51:03,"[[26, Fingers], [36, Cross], [38, Crossed], [82, Congrats], [102, Sofia], [110, Sofia Vergara], [125, Support], [136, Actress]]","[[110, Sofia Vergara]]","[RT, @, COVERGIRL, RT, COVERGIRL, Sofia, Vergara, –, Best, Supporting, Actress, PGRedCarpet]"
1161,"It's time for the best Golden Globes ever! @NotTinaFey &amp; Amy Poehler are gonna kill it! Lets go @bekgaston, @amanda_wongg &amp; @JessicaTong!","{'screen_name': 'LMae13', 'id': 24568030}",290622626697736192,2013-01-14 00:53:28,"[[9, It's Time], [22, For the Best], [22, The Best], [29, Golden], [36, Golden Globes], [72, Amy Poehler]]","[[72, Amy Poehler]]","[Globes, @, NotTinaFey, Amy, Poehler, @, JessicaTong]"
1501,@GoldenGlobes Anne Hathaway best supporting Actress &gt;&gt;&gt;,"{'screen_name': 'DerekBiaselli', 'id': 349959219}",290623059939971072,2013-01-14 00:55:11,"[[27, Anne Hathaway], [40, Support], [51, Actress]]","[[27, Anne Hathaway]]","[GoldenGlobes, Anne, Hathaway, Actress]"
1528,RT @ANCALERTS: MT @LexiSchulze Best supporting actress nominee Sofia Vergara is rocking a beautiful black Michael Cinco number at the #GoldenGlobes!,"{'screen_name': 'yeohrocks', 'id': 38415624}",290656197793697792,2013-01-14 03:06:52,"[[43, Support], [54, Actress], [68, Sofia], [76, Sofia Vergara], [99, Beautiful], [105, Black], [113, Michael], [119, Cinco], [126, Number]]","[[76, Sofia Vergara], [119, Michael Cinco]]","[RT, @, ANCALERTS, MT, @, LexiSchulze, Best, Sofia, Vergara, Michael, Cinco, GoldenGlobes]"
...,...,...,...,...,...,...,...
61178,Best Golden Globes ever. So much love for Adele and Jennifer Lawrence #GirlCrush,"{'screen_name': 'MinaKokic', 'id': 46554204}",290675665471160321,2013-01-14 04:24:13,"[[11, Golden], [18, Golden Globes], [47, Adele], [57, Jenni], [60, Jennifer], [69, Jennifer Lawrence], [69, Lawrence]]","[[69, Jennifer Lawrence], [47, Adele]]","[Best, Golden, Globes, Adele, Jennifer, Lawrence, GirlCrush]"
61197,"RT @SandraOldfield: Ironic that the best acceptance speech for an acting awards show came from a singer, Adele #goldenglobes","{'screen_name': 'jeffbruchesi', 'id': 208374605}",290675667958370304,2013-01-14 04:24:14,"[[40, The Best], [47, Accept], [51, Acceptance], [58, Speech], [72, Acting], [79, Awards], [103, Singer], [110, Adele]]","[[110, Adele]]","[RT, @, SandraOldfield, Adele]"
61205,"""@goldenglobes: Best Original Song - ""Skyfall"" - Adele (@OfficialAdele) &amp; Paul Epworth (@PaulEpworth) - Skyfall - #GoldenGlobes"" :o","{'screen_name': 'radarkepo', 'id': 141178894}",290675671863283712,2013-01-14 04:24:15,"[[27, Origin], [29, Original], [34, Original Song], [54, Adele], [115, Skyfall]]","[[90, Paul Epworth], [54, Adele]]","[Best, Original, Song, Adele, @, OfficialAdele, Paul, Epworth, @, PaulEpworth, GoldenGlobes]"
61252,"RT @televisionary: Best Supporting Actress, TV goes to… Maggie Smith. As always. #GoldenGlobes","{'screen_name': 'edgicovi', 'id': 44584283}",290675714229944320,2013-01-14 04:24:25,"[[31, Support], [42, Actress], [61, Maggi], [62, Maggie], [68, Smith], [79, Always]]","[[68, Maggie Smith]]","[RT, @, Actress, Maggie, Smith, GoldenGlobes]"


In [23]:
filtered_df_best_words[['text']]

,text
663,RT @TheAdvocateMag: Modern Family is up for best TV comedy and whose Eric Stonestreet (who plays gay dad Cam) is up for best supporting actor. #GoldenGlobes
890,RT @COVERGIRL: Our fingers are crossed! #goldenglobes RT @savvysassymoms: Congrats to @COVERGIRL Sofia Vergara – Best Supporting Actress #PGRedCarpet
1161,"It's time for the best Golden Globes ever! @NotTinaFey &amp; Amy Poehler are gonna kill it! Lets go @bekgaston, @amanda_wongg &amp; @JessicaTong!"
1501,@GoldenGlobes Anne Hathaway best supporting Actress &gt;&gt;&gt;
1528,RT @ANCALERTS: MT @LexiSchulze Best supporting actress nominee Sofia Vergara is rocking a beautiful black Michael Cinco number at the #GoldenGlobes!
...,...
61178,Best Golden Globes ever. So much love for Adele and Jennifer Lawrence #GirlCrush
61197,"RT @SandraOldfield: Ironic that the best acceptance speech for an acting awards show came from a singer, Adele #goldenglobes"
61205,"""@goldenglobes: Best Original Song - ""Skyfall"" - Adele (@OfficialAdele) &amp; Paul Epworth (@PaulEpworth) - Skyfall - #GoldenGlobes"" :o"
61252,"RT @televisionary: Best Supporting Actress, TV goes to… Maggie Smith. As always. #GoldenGlobes"


In [39]:
#tried grouping similar tweets

# def assign_groups_to_tweets(df, trigrams, threshold=70):
#     groups = defaultdict(list)
#     group_counter = 0
    
#     for index, row in df.iterrows():
#         tweet = row['text'].lower()
#         tweet = re.sub(r'[^a-zA-Z0-9\s]', ' ', tweet)
#         for trigram in trigrams:
#             tri_text = ' '.join(trigram[0])
#             if tri_text in tweet:
#                 #print(tri_text,tweet)
#                 candidate = ' '.join(tweet.split()[tweet.split().index(trigram[0][0]):])
#                 added = False
#                 for key in groups.keys():
#                     if fuzz.partial_ratio(candidate, key) > threshold:
#                         groups[key].append(candidate)
#                         df.at[index, 'group'] = key
#                         added = True
#                         break
#                 if not added:
#                     group_counter += 1
#                     group_key = f'Group_{group_counter}'
#                     groups[group_key].append(candidate)
#                     df.at[index, 'group'] = group_key

#     return df, groups




In [40]:
# filtered_tweets = filtered_df_best_words.copy()
# filtered_tweets['text'] = filtered_tweets['text'].apply(lambda x: x.lower())
# filtered_tweets['group'] = None 

# trigrams = top_best_bigrams

# filtered_tweets, groups = assign_groups_to_tweets(filtered_tweets, trigrams)


# # print(filtered_tweets.head())
# for key, group in groups.items():
#     print(f"{key}:")
#     for candidate in set(group):
#         print(f"  - {candidate}")

Group_1:
  - best tv comedy and whose eric stonestreet who plays gay dad cam is up for best supporting actor goldenglobes
Group_2:
  - best supporting actress pgredcarpet
Group_3:
  - best golden globes ever nottinafey amp amy poehler are gonna kill it lets go bekgaston amanda wongg amp jessicatong
Group_4:
  - best supporting actress gt gt gt
Group_5:
  - best supporting actress nominee sofia vergara is rocking a beautiful black michael cinco number at the goldenglobes
Group_6:
  - best supporting actress i will literally get on a plane to la and stab helen hunt in the tit goldenglobes
Group_7:
  - best golden globes entrance by far rt jenspace a car just ran into the apple store in lincoln park http t co 7k55a8m4
Group_8:
  - best hosts ever goldenglobes
Group_9:
  - best hosts ever goldenglobes
Group_10:
  - best hosts ever goldenglobes
Group_11:
  - best hosts ever goldenglobes
Group_12:
  - best hosts ever goldenglobes
Group_13:
  - best supporting actor in a drama goldenglobes
Gr

In [11]:
def assign_trigrams_to_tweets(df, trigrams):
    df['matching_trigram'] = None  
    
    for index, row in df.iterrows():
        tweet = row['text'].lower()
        for trigram in trigrams:
            tri_text = ' '.join(trigram[0])
            if tri_text in tweet:
                df.at[index, 'matching_trigram'] = tri_text
                break 

    return df

In [12]:
grouped_awards = assign_trigrams_to_tweets(filtered_df_best_words,top_best_bigrams)

<ipython-input-11-f9e2e7c869d2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['matching_trigram'] = None


In [13]:
grouped_awards

,text,user,id,timestamp_ms,movie,name,NLTK,matching_trigram
663,RT @TheAdvocateMag: Modern Family is up for best TV comedy and whose Eric Stonestreet (who plays gay dad Cam) is up for best supporting actor. #GoldenGlobes,"{'screen_name': 'sandrasick', 'id': 19883537}",290621553153036289,2013-01-14 00:49:12,"[[33, Modern Family], [33, Family], [58, Comedy], [79, Stone], [80, Stones], [85, Eric Stonestreet], [132, Support], [141, Best Supporting Actor], [141, Actor]]","[[85, Eric Stonestreet]]","[RT, @, TheAdvocateMag, Family, Eric, Stonestreet, Cam, GoldenGlobes]",best supporting actor
890,RT @COVERGIRL: Our fingers are crossed! #goldenglobes RT @savvysassymoms: Congrats to @COVERGIRL Sofia Vergara – Best Supporting Actress #PGRedCarpet,"{'screen_name': 'HannaGillispie', 'id': 62957506}",290622018582360064,2013-01-14 00:51:03,"[[26, Fingers], [36, Cross], [38, Crossed], [82, Congrats], [102, Sofia], [110, Sofia Vergara], [125, Support], [136, Actress]]","[[110, Sofia Vergara]]","[RT, @, COVERGIRL, RT, COVERGIRL, Sofia, Vergara, –, Best, Supporting, Actress, PGRedCarpet]",best supporting actress
1161,"It's time for the best Golden Globes ever! @NotTinaFey &amp; Amy Poehler are gonna kill it! Lets go @bekgaston, @amanda_wongg &amp; @JessicaTong!","{'screen_name': 'LMae13', 'id': 24568030}",290622626697736192,2013-01-14 00:53:28,"[[9, It's Time], [22, For the Best], [22, The Best], [29, Golden], [36, Golden Globes], [72, Amy Poehler]]","[[72, Amy Poehler]]","[Globes, @, NotTinaFey, Amy, Poehler, @, JessicaTong]",best golden globes
1501,@GoldenGlobes Anne Hathaway best supporting Actress &gt;&gt;&gt;,"{'screen_name': 'DerekBiaselli', 'id': 349959219}",290623059939971072,2013-01-14 00:55:11,"[[27, Anne Hathaway], [40, Support], [51, Actress]]","[[27, Anne Hathaway]]","[GoldenGlobes, Anne, Hathaway, Actress]",best supporting actress
1528,RT @ANCALERTS: MT @LexiSchulze Best supporting actress nominee Sofia Vergara is rocking a beautiful black Michael Cinco number at the #GoldenGlobes!,"{'screen_name': 'yeohrocks', 'id': 38415624}",290656197793697792,2013-01-14 03:06:52,"[[43, Support], [54, Actress], [68, Sofia], [76, Sofia Vergara], [99, Beautiful], [105, Black], [113, Michael], [119, Cinco], [126, Number]]","[[76, Sofia Vergara], [119, Michael Cinco]]","[RT, @, ANCALERTS, MT, @, LexiSchulze, Best, Sofia, Vergara, Michael, Cinco, GoldenGlobes]",best supporting actress
...,...,...,...,...,...,...,...,...
61178,Best Golden Globes ever. So much love for Adele and Jennifer Lawrence #GirlCrush,"{'screen_name': 'MinaKokic', 'id': 46554204}",290675665471160321,2013-01-14 04:24:13,"[[11, Golden], [18, Golden Globes], [47, Adele], [57, Jenni], [60, Jennifer], [69, Jennifer Lawrence], [69, Lawrence]]","[[69, Jennifer Lawrence], [47, Adele]]","[Best, Golden, Globes, Adele, Jennifer, Lawrence, GirlCrush]",best golden globes
61197,"RT @SandraOldfield: Ironic that the best acceptance speech for an acting awards show came from a singer, Adele #goldenglobes","{'screen_name': 'jeffbruchesi', 'id': 208374605}",290675667958370304,2013-01-14 04:24:14,"[[40, The Best], [47, Accept], [51, Acceptance], [58, Speech], [72, Acting], [79, Awards], [103, Singer], [110, Adele]]","[[110, Adele]]","[RT, @, SandraOldfield, Adele]",best acceptance speech
61205,"""@goldenglobes: Best Original Song - ""Skyfall"" - Adele (@OfficialAdele) &amp; Paul Epworth (@PaulEpworth) - Skyfall - #GoldenGlobes"" :o","{'screen_name': 'radarkepo', 'id': 141178894}",290675671863283712,2013-01-14 04:24:15,"[[27, Origin], [29, Original], [34, Original Song], [54, Adele], [115, Skyfall]]","[[90, Paul Epworth], [54, Adele]]","[Best, Original, Song, Adele, @, OfficialAdele, Paul, Epworth, @, PaulEpworth, GoldenGlobes]",best original song
61252,"RT @televisionary: Best Supporting Actress, TV goes to… Maggie Smith. As always. #GoldenGlobes","{'screen_name': 'edgicovi', 'id': 44584283}",290675714229944320,2013-01-14 04:24:25,"[[31, Support], [42, Actress], [61, Maggi], [62, Maggie], [68, Smith

In [74]:
grouped_awards[grouped_awards['matching_trigram']=='best television series'].head(50)

,text,user,id,timestamp_ms,movie,name,NLTK,matching_trigram
12013,RT @goldenglobes: Best Television Series Actor - Drama - Damian Lewis (@Lewis_Damian) - Homeland (@SHO_Homeland) - #GoldenGlobes,"{'screen_name': 'manuelaferri5', 'id': 151819770}",290675890189381633,2013-01-14 04:25:07,"[[32, Televisio], [33, Television], [40, Series], [46, Actor], [54, Drama], [63, Damian], [96, Homeland]]","[[69, Damian Lewis]]","[RT, @, Series, Actor, Drama, Lewis, @, Lewis_Damian, Homeland, @, SHO_Homeland, GoldenGlobes]",best television series
12194,PUMBA! Best Television Series Actor - Drama - Damian Lewis (@Lewis_Damian) - Homeland (@SHO_Homeland) - #GoldenGlobes,"{'screen_name': 'TwitLaz', 'id': 19397151}",290632693547421696,2013-01-14 01:33:28,"[[21, Televisio], [22, Television], [29, Series], [35, Actor], [43, Drama], [52, Damian], [85, Homeland]]","[[58, Damian Lewis]]","[Series, Actor, Drama, Lewis, @, Lewis_Damian, Homeland, @, SHO_Homeland, GoldenGlobes]",best television series
12230,YES!! RT @goldenglobes: Best Television Series Actor - Drama - Damian Lewis (@Lewis_Damian) - Homeland (@SHO_Homeland) - #GoldenGlobes,"{'screen_name': 'riannajanine', 'id': 498299914}",290632719849893890,2013-01-14 01:33:34,"[[38, Televisio], [39, Television], [46, Series], [52, Actor], [60, Drama], [69, Damian], [102, Homeland]]","[[75, Damian Lewis]]","[RT, @, Series, Actor, Drama, Lewis, @, Lewis_Damian, Homeland, @, SHO_Homeland, GoldenGlobes]",best television series
12398,"""@goldenglobes: Best Television Series Actor - Drama - Damian Lewis"" #GoldenGlobes"" he looks like Hugh Laurie. #missinghouse","{'screen_name': 'BooCarter08', 'id': 108665678}",290632783032897537,2013-01-14 01:33:49,"[[30, Televisio], [31, Television], [38, Series], [44, Actor], [52, Drama], [61, Damian], [108, Lauri], [109, Hugh Laurie]]","[[67, Damian Lewis], [109, Hugh Laurie]]","[Series, Actor, Drama, Lewis, GoldenGlobes, Hugh, Laurie]",best television series
12430,“@goldenglobes: Best Television Series Actor - Drama - Damian Lewis (@Lewis_Damian) - Homeland (@SHO_Homeland) - #GoldenGlobes” Yes! Amazing,"{'screen_name': 'taylorvisoski', 'id': 39902190}",290632780046540800,2013-01-14 01:33:49,"[[30, Televisio], [31, Television], [38, Series], [44, Actor], [52, Drama], [61, Damian], [94, Homeland], [140, Amazing]]","[[67, Damian Lewis]]","[@, Series, Actor, Drama, Lewis, @, Lewis_Damian, Homeland, @, SHO_Homeland, GoldenGlobes, ”, Yes]",best television series
12451,“@goldenglobes: Best Television Series Actor - Drama - Damian Lewis (@Lewis_Damian) - Homeland (@SHO_Homeland) - #GoldenGlobes” UHUUUU,"{'screen_name': 'gabichanas', 'id': 17865061}",290632792130338816,2013-01-14 01:33:51,"[[30, Televisio], [31, Television], [38, Series], [44, Actor], [52, Drama], [61, Damian], [94, Homeland]]","[[67, Damian Lewis]]","[@, Series, Actor, Drama, Lewis, @, Lewis_Damian, Homeland, @, SHO_Homeland, GoldenGlobes, ”, UHUUUU]",best television series
12461,YAY RT @goldenglobes: Best Television Series Actor - Drama - Damian Lewis (@Lewis_Damian) - Homeland (@SHO_Homeland) - #GoldenGlobes,"{'screen_name': 'Helen_Flight', 'id': 137111474}",290632791224369152,2013-01-14 01:33:51,"[[36, Televisio], [37, Television], [44, Series], [50, Actor], [58, Drama], [67, Damian], [100, Homeland]]","[[73, Damian Lewis]]","[YAY, RT, @, Series, Actor, Drama, Lewis, @, Lewis_Damian, Homeland, @, SHO_Homeland, GoldenGlobes]",best television series
12472,Great show RT @goldenglobes: Best Television Series Actor - Drama - Damian Lewis (@Lewis_Damian) - Homeland (@SHO_Homeland) - #GoldenGlobes,"{'screen_name': 'JudynTX', 'id': 260911305}",290632790393884672,2013-01-14 01:33:51,"[[5, Great], [44, Televisio], [45, Television], [52, Series], [58, Actor], [66, Drama], [75, Damian], [108, Homeland]]","[[81, Damian Lewis]]","[Great, RT, @, Series, Actor, Drama, Lewis, @, Lewis_Damian, Homeland, @, SHO_Homeland, GoldenGlobes]",best television series
12604,“@goldenglobes: Best Television Series Actor - Drama - Damian L

In [15]:
grouped_awards.groupby('matching_trigram').count()

,text,user,id,timestamp_ms,movie,name,NLTK
matching_trigram,,,,,,,
best acceptance speech,23,23,23,23,23,23,23
best actor drama,6,6,6,6,6,6,6
best actor motion,8,8,8,8,8,8,8
best actor tv,19,19,19,19,19,19,19
best actress comedy,8,8,8,8,8,8,8
best actress motion,9,9,9,9,9,9,9
best actress tv,13,13,13,13,13,13,13
best animated feature,9,9,9,9,9,9,9
best animated film,3,3,3,3,3,3,3


In [86]:
def remove_strings(list_to_modify, reference_list, special_string):
    special_string = special_string.lower()
    
    list_to_modify = [string for string in list_to_modify 
                      if string.lower() not in (ref_string.lower() for ref_string in reference_list)]
    
    list_to_modify = [string for string in list_to_modify 
                      if special_string.find(string.lower()) == -1]
    
    return list_to_modify

In [104]:
def remove_actor_names(list_to_modify, actors_list):
    names_to_remove = []
    for sublist in actors_list:
        for _, name in sublist:
            if len(name.split())==2:
                first_name, last_name = name.split()
                names_to_remove.extend([first_name.lower(), last_name.lower(), name.lower().replace(" ", "")])
            elif len(name.split())==1:
                first_name = name.split()[0]
                names_to_remove.extend([first_name.lower()])
            
    list_to_modify = [string for string in list_to_modify
                      if string.lower() not in names_to_remove]
    
    return list_to_modify

In [157]:
def find_winner(df,movie_or_actor,award_name):
    actor_names = []


    for index, row in df.iterrows():
        actors_list = row[movie_or_actor]
        for actor in actors_list:
            name = actor[1]
            actor_names.append(name)
    #print(actor_names)
    if movie_or_actor=='movie':
        ref_list=['Drama','actor']
        actor_names=remove_strings(actor_names,ref_list,award_name)
        actor_names=remove_actor_names(actor_names,df['name'])
    #print(actor_names)  
    actor_counts = Counter(actor_names)

    max_occurrence = max(actor_counts.values())

    most_frequent_actors = [actor for actor, count in actor_counts.items() if count == max_occurrence]
    
    filtered_list=[]
    for string in most_frequent_actors:
        if not any(string in s for s in most_frequent_actors if string != s):
            filtered_list.append(string)
    return filtered_list

In [158]:
winner = find_winner(grouped_awards[grouped_awards['matching_trigram']=='best animated film'],movie_or_actor='movie',award_name='best animated film')
print(winner)

['Reference', 'Anne Hathaway', 'Exposed', 'Followed', 'Lovely', 'Skewer', 'Brave', 'The Film', 'Disney', 'Released']


In [107]:
def get_presenters(df, keywords=['presenting', 'presenter', 'announce', 'announcing']):
    
    filtered_df = df[df['text'].str.contains('|'.join(keywords), case=False, na=False)]
    
    
    actor_names = set()
    for index, row in filtered_df.iterrows():
        actors_list = row['name']
        for actor in actors_list:
            name = actor[1]
            actor_names.add(name)
            
    return list(actor_names)



In [135]:

presenters = get_presenters(grouped_awards[grouped_awards['matching_trigram']=='best actor tv'])
print(presenters)

[]


In [171]:
def get_nominees(df,movie_or_actor, keywords=['nominee','nominees','nominated'],winner=[''],award_name=''):
    
    filtered_df = df[df['text'].str.contains('|'.join(keywords), case=False, na=False)]
    
    filtered_df = filtered_df[filtered_df['text'].str.contains('|'.join([award_name]), case=False, na=False)]
    
    actor_names = set()
    for index, row in filtered_df.iterrows():
        actors_list = row[movie_or_actor]
        for actor in actors_list:
            name = actor[1]
            actor_names.add(name)
    #print(actor_names)
    if movie_or_actor=='movie':
        ref_list=['Drama','actor']
        actor_names=remove_strings(actor_names,ref_list,award_name)
        actor_names=remove_actor_names(actor_names,df['name'])
    
    return list(actor_names)

In [172]:
print(get_nominees(df_movie_actor,movie_or_actor='name',winner=winner,award_name='best actor tv'))

['Damian Lewis']


['emily blunt', 'judi dench', 'maggie smith', 'meryl streep']

In [122]:
def audit_nominees(df, keywords=['presenting', 'presenter', 'announce', 'announcing']):
    
    filtered_df = df[df['text'].str.contains('|'.join(keywords), case=False, na=False)]
    return filtered_df

In [143]:
temp_df=audit_nominees(df_movie_actor,["lucy liu", "debra messing"])
temp_df.head(50)

,text,user,id,timestamp_ms,movie,name,NLTK
176,Lucy Liu is a bad mamma jamma #GoldenGlobes,"{'screen_name': 'MsRaVyne', 'id': 27525210}",290620878616686593,2013-01-14 00:46:31,"[[23, Mamma]]","[[8, Lucy Liu]]","[Lucy, Liu, GoldenGlobes]"
417,"RT @Possessionista: Okay, I know half of you are going to freak out...but I kind of love Lucy Liu's dress. #GoldenGlobes","{'screen_name': 'JessicaOglesbee', 'id': 302689151}",290621306095927297,2013-01-14 00:48:13,"[[32, I Know], [63, Freak], [67, Freak Out], [88, Of Love]]","[[97, Lucy Liu]]","[RT, @, Possessionista, Okay, Lucy, Liu, GoldenGlobes]"
580,Best so far #goldenglobes #RedCarpet Anne Hathaway. Lucy Liu. And always a fav #Nicole Kidman,"{'screen_name': 'the_momcierge', 'id': 49670781}",290621517312688128,2013-01-14 00:49:03,"[[50, Anne Hathaway], [74, Always]]","[[50, Anne Hathaway], [61, Lucy Liu]]","[Best, RedCarpet, Anne, Hathaway, Lucy, Liu, Nicole, Kidman]"
746,RT @JasonBinn: Smash's Debra Messing wearing a beautiful dkny dress @goldenglobes #redcarpet #hfpa http://t.co/4YnkaPWs,"{'screen_name': 'jackiefuentes', 'id': 19691949}",290623684413112321,2013-01-14 00:57:40,"[[20, Smash], [28, Debra], [36, Debra Messing], [36, Messing], [56, Beautiful]]","[[36, Debra Messing]]","[RT, @, JasonBinn, Smash, Debra, Messing, @]"
886,Lucy Liu modeló un vestido muy floral de Carolina Herrera #GuruRedCarpet #GoldenGlobes #RedCarpetGuru http://t.co/YsezrPe6,"{'screen_name': 'GurudelEstilo', 'id': 809260920}",290622013591138305,2013-01-14 00:51:02,"[[14, Model], [46, Carol], [49, Carolina], [56, Herrer], [57, Carolina Herrera]]","[[8, Lucy Liu], [57, Carolina Herrera]]","[Lucy, Liu, Carolina, Herrera, GuruRedCarpet, GoldenGlobes, RedCarpetGuru]"
888,"Sorry Lucy Liu, but nobody wears a curtain like Carol Burnett. #GoldenGlobes http://t.co/Rj0FqNKW","{'screen_name': '57MCM', 'id': 121594383}",290622012563529728,2013-01-14 00:51:02,"[[5, Sorry], [26, Nobody], [53, Carol], [61, Carol Burnett]]","[[14, Lucy Liu], [61, Carol Burnett]]","[Sorry, Lucy, Liu, Carol, Burnett, GoldenGlobes]"
908,Debra Messing... why? why??? #GoldenGlobes,"{'screen_name': 'CookieCCouture', 'id': 244739094}",290622140158455808,2013-01-14 00:51:32,"[[5, Debra], [13, Debra Messing], [13, Messing]]","[[13, Debra Messing]]","[Debra, Messing, GoldenGlobes]"
1075,Lucy Liu: amazing. WINNING. #GoldenGlobes,"{'screen_name': 'nikkistrick', 'id': 5689632}",290622544329990144,2013-01-14 00:53:08,"[[17, Amazing], [26, Winning]]","[[8, Lucy Liu]]","[Lucy, Liu, WINNING, GoldenGlobes]"
1105,Loving Lucy liu's hair #eredcarpet #GoldenGlobes,"{'screen_name': 'wabigai', 'id': 429585238}",290622603658412032,2013-01-14 00:53:22,"[[6, Loving]]","[[15, Lucy Liu]]","[Lucy, GoldenGlobes]"
1128,Kind of in love with Lucy Liu's massive floral gown and braid look. #GoldenGlobes,"{'screen_name': 'designcrush', 'id': 19996631}",290622607781404673,2013-01-14 00:53:23,"[[15, In Love], [39, Massive]]","[[29, Lucy Liu]]","[Kind, Lucy, Liu, GoldenGlobes]"


In [47]:
# temp_presenters=get_presenters(temp_df,keywords=['nominee', 'nominees'])
# temp_presenters

['Kristen Wiig',
 'Jessica Chastain',
 'Jennifer Lawrence',
 'Will Ferrell',
 'Anne Hathaway']

In [145]:
award_list=grouped_awards['matching_trigram'].unique()
award_list

array(['best supporting actor', 'best supporting actress',
       'best golden globes', 'best hosts ever', 'best tv series',
       'best original song', 'best actress tv', 'best motion picture',
       'best television series', 'best actor tv', 'best actor drama',
       'best acceptance speech', 'best original score',
       'best speech ever', 'best actress comedy', 'best actress motion',
       'best screenplay quentin', 'best foreign film',
       'best animated feature', 'best animated film', 'best director ben',
       'best comedy series', 'best actor motion'], dtype=object)

In [147]:
for i in award_list:
    print(i)

best supporting actor
best supporting actress
best golden globes
best hosts ever
best tv series
best original song
best actress tv
best motion picture
best television series
best actor tv
best actor drama
best acceptance speech
best original score
best speech ever
best actress comedy
best actress motion
best screenplay quentin
best foreign film
best animated feature
best animated film
best director ben
best comedy series
best actor motion


In [173]:
awards={}
potential_movie_awards=['series','film']

for award_name in award_list:
    movie_or_actor="name"
    if any(word in potential_movie_awards for word in award_name.split()):
        movie_or_actor="movie"
        
    winner = find_winner(grouped_awards[grouped_awards['matching_trigram']==award_name],movie_or_actor=movie_or_actor,award_name=award_name)
    winner_dict={}
    winner_dict["winner"]=winner
    awards[award_name]=winner_dict
    
    nominee=get_nominees(df_movie_actor,movie_or_actor=movie_or_actor,winner=winner,award_name=award_name)
    nominee_dict={}
    nominee_dict["nominees"]=nominee
    awards[award_name].update(nominee_dict)
    
    presenters=get_presenters(grouped_awards[grouped_awards['matching_trigram']==award_name])
    presenter_dict={}
    presenter_dict["Presenter"]=presenters
    awards[award_name].update(presenter_dict)
    

In [174]:
awards

{'best supporting actor': {'winner': ['Christoph Waltz'],
  'nominees': ['Kate Hudson', 'Bradley Cooper'],
  'Presenter': []},
 'best supporting actress': {'winner': ['Anne Hathaway'],
  'nominees': ['Sofia Vergara',
   'Selena Gomez',
   'Mary Todd Lincoln',
   'Les Miserables',
   'Anne Hathaway',
   'Michael Cinco'],
  'Presenter': ['Dennis Quaid', 'Kerry Washington']},
 'best golden globes': {'winner': ['Tina Fey'],
  'nominees': [],
  'Presenter': ['Bill Clinton',
   'Tina Fey',
   'Clinton P',
   'Amy Poehler',
   'Amy A',
   'Tina Ho']},
 'best hosts ever': {'winner': ['Amy Poehler'],
  'nominees': [],
  'Presenter': ['Tina Fey']},
 'best tv series': {'winner': ['Comedy', 'Lena Dunham'],
  'nominees': ['The Star',
   'Actress',
   'Winner',
   'Comedy',
   'Lena Dunham',
   'Thanks'],
  'Presenter': ['Jimmy Fallon', 'Jay Leno']},
 'best original song': {'winner': ['Adele'],
  'nominees': ['Jon Bon Jovi', 'Keith Urban', 'Song Li', 'Adele', 'Haha'],
  'Presenter': ['J. Lo']},
 'be